# Proyección promedio en stock (días)

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "pachuca" in nombre and 'alfa' in nombre 
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca',
 'df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'publicado_hace':'tiempo_de_publicacion','tiempo_publicacion':'tiempo_de_publicacion','Tiempo_publicacion':'tiempo_de_publicacion',}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['tiempo_de_publicacion']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

Columnas renombradas en los DataFrames filtrados.


In [3]:
import re

# Función para extraer mes y año del nombre del DataFrame
def extraer_mes_y_ano(nombre):
    # Expresión regular para capturar tanto meses completos como abreviados
    match = re.search(r'(ene|feb|mar|abr|may|jun|jul|ago|sep|oct|nov|dic|enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)_(\d{4})', nombre)
    if match:
        mes = match.group(1).lower()  # Mes en texto
        ano = int(match.group(2))  # Año en formato numérico
        return mes, ano
    return None, None

# Diccionario de traducción de meses de texto a números
meses_dict = {
    'ene': 1, 'feb': 2, 'mar': 3, 'abr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'ago': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dic': 12,
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
    'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

df_resultados = pd.DataFrame(columns=['tiempo_de_publicacion', 'mes', 'año',])
for nombre_df in nombres_df_filtrados:
    df = globals()[nombre_df]
    
    # Extraer mes y año del nombre del DataFrame
    mes_str, ano = extraer_mes_y_ano(nombre_df)
    
    # Verificar que se haya extraído el mes y el año correctamente
    if mes_str is not None and ano is not None:
        mes = meses_dict.get(mes_str)  # Convertir el nombre del mes en su número correspondiente
        
        # Añadir las columnas de mes y año al DataFrame actual
        df['mes'] = mes
        df['año'] = ano
        # Excluir entradas vacías o todas NA antes de la concatenación
        df = df.dropna(how='all', axis=1)
        df_resultados = pd.concat([df_resultados, df], ignore_index=True)
df_resultados = df_resultados[['mes','año','tiempo_de_publicacion']]
df_resultados


,mes,año,tiempo_de_publicacion
0,4,2024,10
1,4,2024,15
2,4,2024,6
3,4,2024,7
4,4,2024,7
...,...,...,...
21553,9,2023,NaN
21554,9,2023,NaN
21555,9,2023,NaN
21556,9,2023,NaN


In [7]:
df_alfa_agosto_2024_pachuca['tiempo_de_publicacion'].value_counts()

tiempo_de_publicacion
7     277
3     161
2     126
16    126
1     112
4     100
6      87
5      76
10     73
9      59
8      39
11     31
14     29
15     23
21     21
13     19
12     17
17     16
18      7
22      4
19      3
Name: count, dtype: int64

In [21]:
promedios_df = df_resultados.groupby(['mes', 'año'])['tiempo_de_publicacion'].mean().astype(int).reset_index()
meses_dict_invertido = {v: k for k, v in meses_dict.items()}
promedios_df['label'] = promedios_df['mes'].apply(lambda x: meses_dict_invertido[x]) + '-' + promedios_df['año'].astype(str)
promedios_df = promedios_df.sort_values(by=['año', 'mes'])
promedios_df

,mes,año,tiempo_de_publicacion,label
12,10,2022,0,octubre-2022
13,11,2022,0,noviembre-2022
0,2,2023,7,febrero-2023
2,3,2023,33,marzo-2023
6,6,2023,0,junio-2023
8,7,2023,3,julio-2023
11,9,2023,3,septiembre-2023
1,2,2024,11,febrero-2024
3,3,2024,10,marzo-2024
4,4,2024,10,abril-2024


In [22]:
df_promedios = promedios_df[promedios_df['tiempo_de_publicacion'] != 0]

In [24]:
import os
import plotly.graph_objects as go
import plotly.io as pio

fig = go.Figure()
fig.add_trace(go.Scatter(
        x=df_promedios['label'], 
        y=df_promedios['tiempo_de_publicacion'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto        
        text=df_promedios['tiempo_de_publicacion'],#.apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Mes-Año",
    yaxis_title="Días",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10),
    #yaxis=dict(
    #    ticklen=10,
    #    tick0=1,
    #    dtick=1,
    #    range=[0,35]
   # ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_proyeccion_dias', carpeta='assets/graficas')
fig.show()

# Por base directa

In [2]:
# Directorio donde se encuentran los archivos CSV (ruta relativa)
directorio_csv = os.path.join(directorio_actual, '../../db/Scrapining/ciudades/pachuca/clean')

# Archivos CSV específicos a cargar
archivos_csv_especificos = ['abril_2024_1.csv', 'agosto_2024_1.csv', 'febrero_2024_1.csv', 'julio_2024_1.csv', 'junio_2024_1.csv', 'mayo_2024_1.csv']

# Cargar los archivos CSV específicos, asignar 'mes' y 'año' a cada registro, y concatenarlos en un solo DataFrame
df_concatenado = pd.concat([pd.read_csv(os.path.join(directorio_csv, archivo)).assign(mes=archivo.split('_')[0], año=archivo.split('_')[1]) for archivo in archivos_csv_especificos], ignore_index=True)
# Filtrar solo las columnas 'mes', 'año', y 'dias_transcurridos'
df_concatenado = df_concatenado[['mes', 'año', 'dias_transcurridos']]
df_concatenado

,mes,año,dias_transcurridos
0,abril,2024,117
1,abril,2024,265
2,abril,2024,11
3,abril,2024,38
4,abril,2024,52
...,...,...,...
10175,mayo,2024,169
10176,mayo,2024,0
10177,mayo,2024,455
10178,mayo,2024,0


In [11]:
meses = {'abril': 4, 'agosto': 8, 'febrero': 2, 'julio': 7, 'junio': 6, 'mayo': 5}
promedios_df = df_concatenado.groupby(['mes', 'año'])['dias_transcurridos'].mean().astype(int).reset_index()
promedios_df['label'] = promedios_df['mes'] + '-' + promedios_df['año'].astype(str)
promedios_df['mes_int'] = promedios_df['mes'].map(meses)
promedios_df = promedios_df.sort_values(by=['año', 'mes_int']).drop('mes_int', axis=1)
promedios_df

,mes,año,dias_transcurridos,label
2,febrero,2024,279,febrero-2024
0,abril,2024,149,abril-2024
5,mayo,2024,252,mayo-2024
4,junio,2024,180,junio-2024
3,julio,2024,279,julio-2024
1,agosto,2024,197,agosto-2024


In [12]:
import os
import plotly.graph_objects as go
import plotly.io as pio

fig = go.Figure()
fig.add_trace(go.Scatter(
        x=promedios_df['label'], 
        y=promedios_df['dias_transcurridos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto        
        text=promedios_df['dias_transcurridos'],#.apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Mes-Año",
    yaxis_title="Días",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10),
    #yaxis=dict(
    #    ticklen=10,
    #    tick0=1,
    #    dtick=1,
    #    range=[0,35]
   # ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_proyeccion_dias', carpeta='assets/graficas')
fig.show()